# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [61]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [67]:
# Constants

#OLLAMA_API = "http://localhost:11434/api/chat"
# 127.0.0.1:11434
#OLLAMA_API = "http://127.0.0.1:11434/api/chat"
OLLAMA_API = "http://host.docker.internal:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "deepseek-r1:7b"
MODEL = "mistral"

In [68]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [69]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [70]:
# Let's just make sure the model is loaded

#!ollama pull llama3.2

In [71]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)  
print(response.json()['message']['content'])

 1. Content Generation: Generative AI can be used to create a variety of content such as blog posts, articles, social media updates, and product descriptions. This can help businesses save time and resources while maintaining consistent quality in their content production.

2. Personalized Marketing: By analyzing customer data, generative AI can create personalized marketing messages for individual customers, increasing the likelihood of engagement and conversion. This can lead to more effective and efficient marketing campaigns.

3. Customer Service: Generative AI can be used to power chatbots and virtual assistants, improving the efficiency and effectiveness of customer service. These AI systems can handle a high volume of queries and provide quick, accurate responses, improving customer satisfaction.

4. Product Design: In product design, generative AI can create new designs based on specific parameters or guidelines. This can help businesses innovate more quickly and efficiently, a

# Web Summarizer

In [72]:

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [73]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [87]:
ed = Website("https://edwarddonner.com")
#print(ed.title)
#print(ed.text)

In [75]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [76]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [77]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.

In [78]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [79]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a massive, posi

In [89]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    payload = {
        "model": MODEL,
        "messages": messages_for(website),
        "stream": False
    }
    # response = openai.chat.completions.create(
    #     model = "gpt-4o-mini",
    #     model = MODEL,
    #     messages = messages_for(website)
    # )
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)  
    #return response.choices[0].message.content
    '''
    {'model': 'mistral',
    'created_at': '2025-02-08T02:58:44.970281Z',
    'message': {'role': 'assistant',
    'content': ' 1. Content Generation: Generative AI can create text, images, and even music, making it invaluable for businesses looking to produce high-quality content at scale. This could include blog posts, social media updates, or ad copy.\n\n2. Personalized Marketing: By analyzing user data, generative AI can generate personalized marketing messages for individual customers, increasing engagement and conversion rates.\n\n3. Product Design: In the field of product design, generative AI can be used to create multiple design options based on specific parameters, helping companies innovate faster and more efficiently.\n\n4. Customer Service: Generative AI can power chatbots that understand and respond to customer inquiries in a natural, human-like way, improving customer service efficiency and satisfaction.\n\n5. Financial Modeling: In finance, generative AI can be used to create synthetic financial data for testing models or analyzing risk, without the need for real-world data that may not be available or could compromise privacy.\n\n6. Drug Discovery: In pharmaceuticals, generative AI is being used to design new molecules for potential drugs, speeding up the drug discovery process and reducing costs.\n\n7. Predictive Maintenance: In manufacturing and maintenance, generative AI can predict when equipment might fail or require maintenance based on patterns in sensor data, preventing downtime and increasing efficiency.\n\n8. Art and Design: Generative AI can be used to create unique artworks, designs for fashion, or even architectural plans, adding a touch of creativity and originality to various industries.\n\n9. Gaming and Entertainment: In gaming and entertainment, generative AI can create new levels, characters, or stories, keeping players engaged and immersed.\n\n10. Data Augmentation: Generative AI can generate synthetic data to augment existing datasets, helping businesses overcome issues of data scarcity and bias in machine learning models.'},
    'done_reason': 'stop',
    'done': True,
    'total_duration': 12884129250,
    'load_duration': 19040833,
    'prompt_eval_count': 15,
    'prompt_eval_duration': 851197000,
    'eval_count': 405,
    'eval_duration': 12015648000}
    '''
    return response.json()['message']['content']

In [90]:
#summarize("https://edwarddonner.com")
summarize("https://joshjanzen.com")

' This website, titled "Josh Janzen - AI & Data Science Leader", belongs to Josh Janzen, a Data Science Director and member of the GenAI Center of Excellence. The primary focus of his work is transforming business visions into measurable outcomes through AI and analytics initiatives.\n\nJosh has led various projects that showcase his expertise in AI and data science, including:\n\n1. **AI Roster Optimization**: A comparative study between OpenAI\'s o1 model and Google OR-Tools for optimizing basketball team rotations. The project featured an interactive web interface using Streamlit, complex constraint handling, and a visual rotation schedule representation.\n\n2. **AI-Powered Math Learning Assistant**: Development of an intelligent math tutoring system using LangGraph for reliable and controlled question generation workflows. The key features include a grade-adaptive questioning system, sophisticated state machine for question generation, comprehensive validation workflow, and real-ti

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [59]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

ConnectionError: Failed to connect to Ollama. Please check that Ollama is downloaded, running and accessible. https://ollama.com/download

## Alternative approach - using OpenAI python library to connect to Ollama

In [53]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

APIConnectionError: Connection error.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [40]:
!ollama pull deepseek-r1:1.5b

/bin/bash: line 1: ollama: command not found


In [41]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:7b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

APIConnectionError: Connection error.

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.